In [3]:
import os 
from glob import glob 
import pandas as pd 
from functools import reduce 
from xml.etree import ElementTree as et 

In [4]:
xmlfiles = glob(r'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\*.xml')
#data cleaning 
replace_text = lambda x: x.replace('\\','//')
xmlfiles = list(map(replace_text,xmlfiles))

In [5]:
xmlfiles

['C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//001.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//002.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//003.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//004.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//005.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//006.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//007.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//008.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//009.xml',
 'C:////Users////Aditya////Projects////TY projects////datapreparation////data_images//010.xml']

In [6]:
#read xml file and extract details 
tree = et.parse('C://Users//Aditya//Projects//TY projects//datapreparation//data_images//001.xml')
root = tree.getroot()

In [7]:
def extract_text(filename):
    tree = et.parse(filename)  # Use the provided filename
    root = tree.getroot()
    
    # Extract file details
    image_name = root.find('filename').text
    
    # Width and height
    size = root.find('size')
    width = size.find('width').text
    height = size.find('height').text
    parser = []
    # Extract annotations
    objs = root.findall('object')
    for obj in objs:
        name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = bbox.find('xmin').text
        ymin = bbox.find('ymin').text
        xmax = bbox.find('xmax').text
        ymax = bbox.find('ymax').text
        parser.append([name,width,height,xmin,xmax,ymin,ymax ])
    
    return parser


In [8]:
parser_all = list(map(extract_text,xmlfiles))

In [9]:
data = reduce(lambda x,y : x+y,parser_all)

In [10]:
df = pd.DataFrame(data,columns = ['name','width','height','xmin','xmax','ymin','ymax'])


In [11]:
df.head()

,name,width,height,xmin,xmax,ymin,ymax
0,car,1920,1200,88,1770,128,1082
1,car,1280,821,17,391,387,641
2,car,1280,821,321,733,405,615
3,car,1280,821,619,1027,408,586
4,car,1280,821,880,1280,405,583


In [12]:
df.shape

(24, 7)

In [13]:
df['name'].value_counts()


person      8
car         5
horse       3
bike        2
dog         1
bicycle     1
giraffe     1
zebra       1
elephant    1
eagle       1
Name: name, dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    24 non-null     object
 1   width   24 non-null     object
 2   height  24 non-null     object
 3   xmin    24 non-null     object
 4   xmax    24 non-null     object
 5   ymin    24 non-null     object
 6   ymax    24 non-null     object
dtypes: object(7)
memory usage: 1.4+ KB


In [15]:
# Convert columns to numeric type
df['xmax'] = pd.to_numeric(df['xmax'])
df['xmin'] = pd.to_numeric(df['xmin'])
df['ymax'] = pd.to_numeric(df['ymax'])
df['ymin'] = pd.to_numeric(df['ymin'])
df['width'] = pd.to_numeric(df['width'])
df['height'] = pd.to_numeric(df['height'])

In [16]:
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
                  
df['w'] = (df['xmax'] - df['xmin']) / df['width']

df['h'] = (df['ymax'] - df['ymin']) / df['height']

In [17]:
df.head()

,name,width,height,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,car,1920,1200,88,1770,128,1082,0.483854,0.504167,0.876042,0.795000
1,car,1280,821,17,391,387,641,0.159375,0.626066,0.292187,0.309379
2,car,1280,821,321,733,405,615,0.411719,0.621194,0.321875,0.255786
3,car,1280,821,619,1027,408,586,0.642969,0.605359,0.318750,0.216809
4,car,1280,821,880,1280,405,583,0.843750,0.601705,0.312500,0.216809


In [18]:
#Split data into train and test 

In [19]:
df['name']

0          car
1          car
2          car
3          car
4          car
5          dog
6      bicycle
7      giraffe
8        zebra
9     elephant
10       eagle
11       horse
12       horse
13       horse
14      person
15      person
16      person
17      person
18      person
19      person
20      person
21      person
22        bike
23        bike
Name: name, dtype: object

In [20]:
print(df.columns)


Index(['name', 'width', 'height', 'xmin', 'xmax', 'ymin', 'ymax', 'center_x',
       'center_y', 'w', 'h'],
      dtype='object')


In [21]:


# Using shape attribute
print(df.shape[0])


24


In [22]:
image = df['name'].unique()

In [23]:
len(image)

10

In [24]:
img_df = pd.DataFrame(image,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])

In [25]:
img_train

('bicycle', 'giraffe', 'dog', 'zebra', 'person', 'eagle', 'bike', 'elephant')

In [26]:
img_test = tuple(img_df.query(f'filename not in{img_train}')['filename'])

In [27]:
len(img_train),len(img_test)

(8, 2)

In [28]:
print(img_test)

('car', 'horse')


In [29]:
print(df.head())  # Inspect the first few rows of the DataFrame
print(df.columns)  # List all columns in the DataFrame


  name  width  height  xmin  xmax  ymin  ymax  center_x  center_y         w  \
0  car   1920    1200    88  1770   128  1082  0.483854  0.504167  0.876042   
1  car   1280     821    17   391   387   641  0.159375  0.626066  0.292187   
2  car   1280     821   321   733   405   615  0.411719  0.621194  0.321875   
3  car   1280     821   619  1027   408   586  0.642969  0.605359  0.318750   
4  car   1280     821   880  1280   405   583  0.843750  0.601705  0.312500   

          h  
0  0.795000  
1  0.309379  
2  0.255786  
3  0.216809  
4  0.216809  
Index(['name', 'width', 'height', 'xmin', 'xmax', 'ymin', 'ymax', 'center_x',
       'center_y', 'w', 'h'],
      dtype='object')


In [30]:
train_df = df.query(f'name in {img_train}')
test_df = df.query(f'name in {img_test}')

In [31]:
train_df.head()

,name,width,height,xmin,xmax,ymin,ymax,center_x,center_y,w,h
5,dog,768,576,120,329,193,556,0.292318,0.650174,0.272135,0.630208
6,bicycle,768,576,115,580,98,434,0.452474,0.461806,0.605469,0.583333
7,giraffe,500,500,153,448,21,458,0.601000,0.479000,0.590000,0.874000
8,zebra,500,500,242,447,203,434,0.689000,0.637000,0.410000,0.462000
9,elephant,5000,4000,156,4856,658,3716,0.501200,0.546750,0.940000,0.764500


In [32]:
test_df.head()

,name,width,height,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,car,1920,1200,88,1770,128,1082,0.483854,0.504167,0.876042,0.795000
1,car,1280,821,17,391,387,641,0.159375,0.626066,0.292187,0.309379
2,car,1280,821,321,733,405,615,0.411719,0.621194,0.321875,0.255786
3,car,1280,821,619,1027,408,586,0.642969,0.605359,0.318750,0.216809
4,car,1280,821,880,1280,405,583,0.843750,0.601705,0.312500,0.216809


In [33]:
#Assign id number to object names
def label_encoding(x):
    label ={'person':0 , 'car':1,'chair':2,'bottle':3,'bottle':3,'elephant':4,'giraff':5,'zebra':6,'horse':7}
    return label.get(x, -1)


In [34]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\Aditya\AppData\Local\Temp\ipykernel_25000\3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_25000\3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [35]:
train_df = train_df.copy()
test_df = test_df.copy()

train_df.loc[:, 'id'] = train_df['name'].apply(label_encoding)
test_df.loc[:, 'id'] = test_df['name'].apply(label_encoding)


In [36]:
train_df.head(10)

,name,width,height,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
5,dog,768,576,120,329,193,556,0.292318,0.650174,0.272135,0.630208,-1
6,bicycle,768,576,115,580,98,434,0.452474,0.461806,0.605469,0.583333,-1
7,giraffe,500,500,153,448,21,458,0.601000,0.479000,0.590000,0.874000,-1
8,zebra,500,500,242,447,203,434,0.689000,0.637000,0.410000,0.462000,6
9,elephant,5000,4000,156,4856,658,3716,0.501200,0.546750,0.940000,0.764500,4
10,eagle,773,512,118,641,37,474,0.490944,0.499023,0.676585,0.853516,-1
14,person,4410,2556,3161,3800,757,1898,0.789229,0.519366,0.144898,0.446401,0
15,person,4410,2556,2750,3103,886,1822,0.663605,0.529734,0.080045,0.366197,0
16,person,4410,2556,432,947,766,1510,0.156349,0.445227,0.116780,0.291080,0
17,person,4410,2556,2147,2461,1128,1536,0.522449,0.521127,0.071202,0.159624,0


In [37]:
#save image and label in text 

In [38]:
import os 
from shutil import move 


In [39]:
train_folder = 'C:\\Users\\Aditya\Projects\\TY projects\\datapreparation\\data_images\\train'
test_folder = 'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\test'

os.mkdir(train_folder)
os.mkdir(test_folder)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\train'

In [ ]:
cols = ['name','id','center_x','center_y','w','h']
groupby_obj_train = train_df[cols].groupby('name')
groupby_obj_test = test_df[cols].groupby('name')


In [40]:
groupby_obj_train.get_group('car').set_index('name').to_csv('smaple.txt',index=False,header=False)

NameError: name 'groupby_obj_train' is not defined

In [41]:
def save_data(filename,folder_path, group_obj):
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path, filename)
    move(src,dst)
    
    text_filename = os.path.join(folder_path,
                                 os.path.splittext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('name').to_csv(text_filename, sep=' ', index=False, header=False)

    
    

In [42]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

NameError: name 'groupby_obj_train' is not defined

In [43]:
filename_series.apply(save_data,args= (train_folder,groupby_obj_train))

NameError: name 'filename_series' is not defined

In [44]:
import os
import shutil

# Function to move a file from source to destination
def move_file(src, dst):
    try:
        # Check if the source file exists
        if os.path.exists(src):
            # Move the file to the destination
            shutil.move(src, dst)
            print(f"File moved successfully from {src} to {dst}")
        else:
            print(f"Error: Source file {src} not found")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
source_file = r'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\002.jpg'  # Update the source file path
destination_file = r'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\train\\002'  # Update the destination file path
move_file(source_file, destination_file)



Error: Source file C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\002.jpg not found


In [45]:
# Example with corrected source file path
source_file = r'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\'
destination_file = r'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\train\\bicycle'
move_file(source_file, destination_file)


An error occurred: Cannot move a directory 'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\' into itself 'C:\\Users\\Aditya\\Projects\\TY projects\\datapreparation\\data_images\\train\\bicycle'.
